#### Learning Selector Group Chat for Dynamic Selection of Agents

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console

from dotenv import load_dotenv

import os

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")
azure_openai_deployment_name = os.getenv(
    "AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo")

if not azure_openai_api_key or \
        not azure_openai_endpoint or \
        not azure_openai_api_version or \
        not azure_openai_deployment_name:
    raise ValueError(
        "Please set Azure Open AI Endpoint Details in your environment variables.")

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
def weather_check_tool(city: str) -> str:
    weather_data = {
        "New York": "It's sunny in New York.",
        "Los Angeles": "It's cloudy in Los Angeles.",
        "Chicago": "It's rainy in Chicago.",
        "London": "It's foggy in London.",
        "Tokyo": "It's sunny in Tokyo.",
        "Sydney": "It's warm in Sydney.",
        "Berlin": "It's chilly in Berlin.",
    }

    return weather_data.get(city, "Weather data not available for this city.")


def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        "USD": {"EUR": 0.85, "JPY": 110.0},
        "EUR": {"USD": 1.18, "JPY": 130.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0077},
        "GBP": {"USD": 1.39, "EUR": 1.16},
        "AUD": {"USD": 0.74, "EUR": 0.63},
    }

    if from_currency in exchange_rates and to_currency in exchange_rates[from_currency]:
        rate = exchange_rates[from_currency][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is {converted_amount:.2f} {to_currency}."
    else:
        return "Currency conversion not available for the specified currencies."

In [ ]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent that plans tasks based on user requests.",
    model_client=model_client,
    system_message="""
        You are a planning agent. Your task is to plan tasks based on user requests.
        Your job is to break down user requests into actionable tasks and assign them to appropriate agents.
        Your Team members are specialized agents that can perform specific tasks.
            - WeatherAgent: Provides weather information for a given city.
            - CurrencyExchangeAgent: Converts currency amounts between different currencies.
        You only plan tasks and do not execute them yourself.
        After all tasks are completed, summarize the findings and end with "TERMINATE".
        
        When assigning tasks, use the following format:
        ```
        1. Agent <agent_name> : Task <task>
        ```
        
        
    """,
)

weather_agent = AssistantAgent(
    "WeatherAgent",
    description="An agent that provides weather information for a given city.",
    model_client=model_client,
    system_message="""
        You are a weather information agent. 
        Your task is to provide weather information for a given city.
        When asked about the weather, respond with the current weather conditions in the specified city.
    """,
    tools=[weather_check_tool],
)

currency_exchange_agent = AssistantAgent(
    "CurrencyExchangeAgent",
    description="An agent that converts currency amounts between different currencies.",
    model_client=model_client,
    system_message="""
        You are a currency exchange agent.
        Your task is to convert currency amounts between different currencies.
        When asked to convert currency, respond with the converted amount in the specified currency.
    """,
    tools=[currency_exchange_tool],
)

In [ ]:
from typing import Sequence


def custom_selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name

    return None

In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(10)
combined_termination = text_mention_termination | max_message_termination

team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_exchange_agent],
    termination_condition=combined_termination,
    allow_repeated_speaker=True,
    selector_func=custom_selector_func,
)

In [ ]:
task = "What's the weather in New York and how much is 100 USD in EUR?"

await Console(
    team.run_stream(task=task))